In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# ! gdown --id 1PUSJ56k93B42XW9oszcPDefM8HeJ2BOg
# ! pip -q uninstall -y kaggle
# ! pip -q install --upgrade pip
# ! pip -q install kaggle --upgrade
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download aishikai/amc-2023

In [3]:
# ! unzip -q /content/amc-2023.zip -d data
# ! rm /content/amc-2023.zip

In [4]:
# !pip install pandarallel

In [5]:
import numpy as np
import polars as pl
import os
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn.functional as F
from tqdm import tqdm
import gc
import multiprocessing
from time import time
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

import random
import gc
tqdm.pandas()
pd.set_option('display.max_rows', 1000)


import warnings
warnings.filterwarnings("ignore")

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
train=pl.read_csv('/content/drive/MyDrive/AMC 2023/train.csv').to_pandas()
test=pl.read_csv('/content/drive/MyDrive/AMC 2023/test.csv').to_pandas()

train['TITLE']=train['TITLE'].fillna('')
test['TITLE']=test['TITLE'].fillna('')

train['DESCRIPTION']=train['DESCRIPTION'].fillna('')
test['DESCRIPTION']=test['DESCRIPTION'].fillna('')

train['BULLET_POINTS']=train['BULLET_POINTS'].fillna('')
test['BULLET_POINTS']=test['BULLET_POINTS'].fillna('')

test['PRODUCT_LENGTH']=np.nan

In [7]:
display(train.head())

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,title_cluster_bert,brand_cluster_bert,bullet_cluster_bert
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,,1650,2125.980000,1772,7782,4511
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",,2755,393.700000,1052,146,2189
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,17,4706,4113
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,1933,736,7176
4,283658,The United Empire Loyalists: A Chronicle of th...,,,6112,598.424000,541,2049,3


In [8]:
import re
def extract_brand_words(text):
    words = re.findall(r'\b\w{1,}\b', text)
    return ' '.join(words[:1])

def extract_bullets_words(text):
    words = re.findall(r'\b\w{1,}\b', text)
    return ' '.join(words[:2])

def extract_desc_words(text):
    words = re.findall(r'\b\w{1,}\b', text)
    return ' '.join(words[:2])


train['BRAND']=train['TITLE'].parallel_apply(extract_brand_words)
test['BRAND']=test['TITLE'].parallel_apply(extract_brand_words)

train['BRAND_BULLET']=train['BULLET_POINTS'].parallel_apply(extract_bullets_words)
test['BRAND_BULLET']=test['BULLET_POINTS'].parallel_apply(extract_bullets_words)

train['BRAND_DESC']=train['DESCRIPTION'].parallel_apply(extract_desc_words)
test['BRAND_DESC']=test['DESCRIPTION'].parallel_apply(extract_desc_words)

train['text']=train['TITLE']+'|'+train['BULLET_POINTS']+'|'+train['DESCRIPTION']
test['text']=test['TITLE']+'|'+test['BULLET_POINTS']+'|'+test['DESCRIPTION']

In [9]:
train['BRAND_PRODUCT_TYPE']=train['BRAND']+'|'+train['PRODUCT_TYPE_ID'].astype(str)
test['BRAND_PRODUCT_TYPE']=test['BRAND']+'|'+test['PRODUCT_TYPE_ID'].astype(str)

In [10]:
train['BRAND_BULLET_PRODUCT_TYPE']=train['BRAND']+"|"+train['BRAND_BULLET']+"|"+train['PRODUCT_TYPE_ID'].astype(str)
test['BRAND_BULLET_PRODUCT_TYPE']=test['BRAND']+"|"+test['BRAND_BULLET']+"|"+test['PRODUCT_TYPE_ID'].astype(str)

In [11]:
train['BRAND_DESC_PRODUCT_TYPE']=train['BRAND']+"|"+train['BRAND_DESC']+"|"+train['PRODUCT_TYPE_ID'].astype(str)
test['BRAND_DESC_PRODUCT_TYPE']=test['BRAND']+"|"+test['BRAND_DESC']+"|"+test['PRODUCT_TYPE_ID'].astype(str)

In [12]:
train['DESC EQUAL BULLET']=(train['BRAND_DESC']==train['BRAND_BULLET'])
test['DESC EQUAL BULLET']=(test['BRAND_DESC']==test['BRAND_BULLET'])

In [13]:
from sklearn.preprocessing import LabelEncoder

train_df=pd.concat([train,test],axis=0)

le=LabelEncoder()
train_df['BRAND']=le.fit_transform(train_df['BRAND'])
train_df['BRAND_BULLET']=le.fit_transform(train_df['BRAND_BULLET'])
train_df['BRAND_DESC']=le.fit_transform(train_df['BRAND_DESC'])
train_df['BRAND_PRODUCT_TYPE']=le.fit_transform(train_df['BRAND_PRODUCT_TYPE'])
train_df['BRAND_BULLET_PRODUCT_TYPE']=le.fit_transform(train_df['BRAND_BULLET_PRODUCT_TYPE'])
train_df['BRAND_DESC_PRODUCT_TYPE']=le.fit_transform(train_df['BRAND_DESC_PRODUCT_TYPE'])
# train_df['text']=le.fit_transform(train_df['text'])

In [14]:
del train_df['TITLE'],train_df['BULLET_POINTS'],train_df['DESCRIPTION']
gc.collect()

0

In [15]:
train=train_df.iloc[:len(train)]
test=train_df.iloc[len(train):]

In [16]:
train['preds']=np.nan
test['preds']=np.nan

In [17]:
d_train=train.copy()

In [18]:
def split_list(numbers, num_splits):
    sublist_len = len(numbers) // num_splits

    # split the list into sublists
    sublists = [[] for i in range(num_splits)]
    for (i, num) in enumerate(numbers):
        sublists[i%num_splits].append(num)
    return sublists

In [19]:
a_train=pd.DataFrame()
a_test=pd.DataFrame()

def get_pred_value(value):
    idx, train, test = value
    for i_,i in enumerate(idx):
        if (i_%1000==0):
            print(f"Done:{i_}/{len(idx)}\n")
        temp=d_train[d_train['BRAND_BULLET_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].values
        ind=d_train[d_train['BRAND_BULLET_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].index

        val2=getVal(temp)
        mape,score2=comp_score(temp,[val2]*len(temp))
    #     preds[ind]=val2
        val=val2
        score=score2
        train.loc[train['BRAND_BULLET_PRODUCT_TYPE']==i,'preds']=min(val,1000)
        test.loc[test['BRAND_BULLET_PRODUCT_TYPE']==i,'preds']=min(val,1000)
    return (train,test)

def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

label_enc=pd.DataFrame(train['BRAND_BULLET_PRODUCT_TYPE'].value_counts())
index=label_enc[label_enc['BRAND_BULLET_PRODUCT_TYPE']>2].index.values.tolist()
# create a pool of worker processes
print(len(index))
indexes = split_list(index,multiprocessing.cpu_count())
train_lists = [train.loc[train['BRAND_BULLET_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
test_lists = [test.loc[test['BRAND_BULLET_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
train_ = train.loc[~train['BRAND_BULLET_PRODUCT_TYPE'].isin(index)]
test_ = test.loc[~test['BRAND_BULLET_PRODUCT_TYPE'].isin(index)]
for i in range(len(indexes)):
    print(f"Shape: {train_lists[i].shape}")

for i in range(len(indexes)):
    print(f"Test Shape: {test_lists[i].shape}")
func_list = [(idx, train_, test_) for (idx, train_, test_) in zip(indexes, train_lists, test_lists)]

if __name__ == '__main__':
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        result = pool.map(get_pred_value, func_list)
    train = pd.concat([result[i][0] for i in range(len(result))], axis = 0, ignore_index = True)
    test = pd.concat([result[i][1] for i in range(len(result))], axis = 0, ignore_index = True)
    train = pd.concat([train, train_], axis = 0, ignore_index = True)
    test = pd.concat([test, test_], axis = 0, ignore_index = True)
    print(train.shape)
    print(test.shape)
    a_train=train[train['preds'].notna()]    
    a_test=test[test['preds'].notna()]
    del train_, test_, train_lists, test_lists, indexes, func_list
    gc.collect()

115264
Shape: (30135, 15)
Shape: (29967, 15)
Shape: (29831, 15)
Shape: (28762, 15)
Shape: (27127, 15)
Shape: (27029, 15)
Shape: (26456, 15)
Shape: (26382, 15)
Shape: (25944, 15)
Shape: (25792, 15)
Shape: (25420, 15)
Shape: (25358, 15)
Shape: (25225, 15)
Shape: (24939, 15)
Shape: (24855, 15)
Shape: (24800, 15)
Shape: (24678, 15)
Shape: (24644, 15)
Shape: (24590, 15)
Shape: (24510, 15)
Shape: (24488, 15)
Shape: (24403, 15)
Shape: (24377, 15)
Shape: (24338, 15)
Shape: (24252, 15)
Shape: (24227, 15)
Shape: (24211, 15)
Shape: (24190, 15)
Shape: (24137, 15)
Shape: (24109, 15)
Shape: (24090, 15)
Shape: (24055, 15)
Shape: (24002, 15)
Shape: (23957, 15)
Shape: (23929, 15)
Shape: (23892, 15)
Shape: (23874, 15)
Shape: (23846, 15)
Shape: (23833, 15)
Shape: (23807, 15)
Test Shape: (9028, 15)
Test Shape: (8793, 15)
Test Shape: (8862, 15)
Test Shape: (8501, 15)
Test Shape: (7948, 15)
Test Shape: (7902, 15)
Test Shape: (7512, 15)
Test Shape: (7543, 15)
Test Shape: (7352, 15)
Test Shape: (7237, 15)
Tes

In [20]:
print(comp_score(train[train['preds'].notna()]['PRODUCT_LENGTH'].values,train[train['preds'].notna()]['preds']))
print(comp_score(a_train[a_train['preds'].notna()]['PRODUCT_LENGTH'].values,a_train[a_train['preds'].notna()]['preds']))
print(len(a_train),len(train))

(0.2759875448053508, 72.40124551946492)
(0.2759875448053508, 72.40124551946492)
1008461 2249686


In [21]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]

def get_pred_value(value):
    idx, train, test = value
    for i_,i in enumerate(idx):
        if (i_%1000==0):
            print(f"Done:{i_}/{len(idx)}\n")
        temp=d_train[d_train['BRAND_DESC_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].values
        ind=d_train[d_train['BRAND_DESC_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].index

        val2=getVal(temp)
        mape,score2=comp_score(temp,[val2]*len(temp))
    #     preds[ind]=val2
        val=val2
        score=score2
        train.loc[train['BRAND_DESC_PRODUCT_TYPE']==i,'preds']=min(val,1000)
        test.loc[test['BRAND_DESC_PRODUCT_TYPE']==i,'preds']=min(val,1000)
    return (train,test)

def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

label_enc=pd.DataFrame(train['BRAND_DESC_PRODUCT_TYPE'].value_counts())
index=label_enc[label_enc['BRAND_DESC_PRODUCT_TYPE']>2].index.values.tolist()
# create a pool of worker processes
print(len(index))
indexes = split_list(index,multiprocessing.cpu_count())
train_lists = [train.loc[train['BRAND_DESC_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
test_lists = [test.loc[test['BRAND_DESC_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
train_ = train.loc[~train['BRAND_DESC_PRODUCT_TYPE'].isin(index)]
test_ = test.loc[~test['BRAND_DESC_PRODUCT_TYPE'].isin(index)]
for i in range(len(indexes)):
    print(f"Shape: {train_lists[i].shape}")

for i in range(len(indexes)):
    print(f"Test Shape: {test_lists[i].shape}")
func_list = [(idx, train_, test_) for (idx, train_, test_) in zip(indexes, train_lists, test_lists)]

if __name__ == '__main__':
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        result = pool.map(get_pred_value, func_list)
    train = pd.concat([result[i][0] for i in range(len(result))], axis = 0, ignore_index = True)
    test = pd.concat([result[i][1] for i in range(len(result))], axis = 0, ignore_index = True)
    train = pd.concat([train, train_], axis = 0, ignore_index = True)
    test = pd.concat([test, test_], axis = 0, ignore_index = True)
    print(train.shape)
    print(test.shape)
    a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
    a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0) 
    del train_, test_, train_lists, test_lists, indexes, func_list
    gc.collect()

25894
Shape: (3660, 15)
Shape: (3390, 15)
Shape: (3351, 15)
Shape: (3331, 15)
Shape: (3291, 15)
Shape: (3287, 15)
Shape: (3285, 15)
Shape: (3280, 15)
Shape: (3274, 15)
Shape: (3267, 15)
Shape: (3262, 15)
Shape: (3257, 15)
Shape: (3255, 15)
Shape: (3239, 15)
Shape: (3231, 15)
Shape: (3231, 15)
Shape: (3227, 15)
Shape: (3221, 15)
Shape: (3216, 15)
Shape: (3214, 15)
Shape: (3208, 15)
Shape: (3207, 15)
Shape: (3203, 15)
Shape: (3199, 15)
Shape: (3197, 15)
Shape: (3197, 15)
Shape: (3195, 15)
Shape: (3194, 15)
Shape: (3191, 15)
Shape: (3191, 15)
Shape: (3187, 15)
Shape: (3185, 15)
Shape: (3183, 15)
Shape: (3178, 15)
Shape: (3178, 15)
Shape: (3176, 15)
Shape: (3174, 15)
Shape: (3171, 15)
Shape: (3167, 15)
Shape: (3167, 15)
Test Shape: (1080, 15)
Test Shape: (1004, 15)
Test Shape: (1012, 15)
Test Shape: (958, 15)
Test Shape: (938, 15)
Test Shape: (924, 15)
Test Shape: (1018, 15)
Test Shape: (907, 15)
Test Shape: (969, 15)
Test Shape: (946, 15)
Test Shape: (976, 15)
Test Shape: (975, 15)
Test S

In [22]:
comp_score(train[train['preds'].notna()]['PRODUCT_LENGTH'].values,train[train['preds'].notna()]['preds'])

(0.40391642594985944, 59.608357405014054)

In [23]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]

def get_pred_value(value):
    idx, train, test = value
    for i_,i in enumerate(idx):
        if (i_%1000==0):
            print(f"Done:{i_}/{len(idx)}\n")
        temp=d_train[d_train['BRAND_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].values
        ind=d_train[d_train['BRAND_PRODUCT_TYPE']==i]['PRODUCT_LENGTH'].index

        val2=getVal(temp)
        mape,score2=comp_score(temp,[val2]*len(temp))
    #     preds[ind]=val2
        val=val2
        score=score2
        train.loc[train['BRAND_PRODUCT_TYPE']==i,'preds']=min(val,1000)
        test.loc[test['BRAND_PRODUCT_TYPE']==i,'preds']=min(val,1000)
    return (train,test)

def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

label_enc=pd.DataFrame(train['BRAND_PRODUCT_TYPE'].value_counts())
index=label_enc[label_enc['BRAND_PRODUCT_TYPE']>1].index.values.tolist()
# create a pool of worker processes
print(len(index))
indexes = split_list(index,multiprocessing.cpu_count())
train_lists = [train.loc[train['BRAND_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
test_lists = [test.loc[test['BRAND_PRODUCT_TYPE'].isin(index_)] for index_ in indexes]
train_ = train.loc[~train['BRAND_PRODUCT_TYPE'].isin(index)]
test_ = test.loc[~test['BRAND_PRODUCT_TYPE'].isin(index)]
for i in range(len(indexes)):
    print(f"Shape: {train_lists[i].shape}")

for i in range(len(indexes)):
    print(f"Test Shape: {test_lists[i].shape}")
func_list = [(idx, train_, test_) for (idx, train_, test_) in zip(indexes, train_lists, test_lists)]

if __name__ == '__main__':
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        result = pool.map(get_pred_value, func_list)
    train = pd.concat([result[i][0] for i in range(len(result))], axis = 0, ignore_index = True)
    test = pd.concat([result[i][1] for i in range(len(result))], axis = 0, ignore_index = True)
    train = pd.concat([train, train_], axis = 0, ignore_index = True)
    test = pd.concat([test, test_], axis = 0, ignore_index = True)
    print(train.shape)
    print(test.shape)
    a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
    a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0) 
    del train_, test_, train_lists, test_lists, indexes, func_list
    gc.collect()

159284
Shape: (10759, 15)
Shape: (10745, 15)
Shape: (10733, 15)
Shape: (10718, 15)
Shape: (10714, 15)
Shape: (10702, 15)
Shape: (10693, 15)
Shape: (10685, 15)
Shape: (10673, 15)
Shape: (10670, 15)
Shape: (10668, 15)
Shape: (10666, 15)
Shape: (10665, 15)
Shape: (10652, 15)
Shape: (10649, 15)
Shape: (10646, 15)
Shape: (10641, 15)
Shape: (10641, 15)
Shape: (10639, 15)
Shape: (10636, 15)
Shape: (10635, 15)
Shape: (10633, 15)
Shape: (10632, 15)
Shape: (10628, 15)
Shape: (10623, 15)
Shape: (10621, 15)
Shape: (10619, 15)
Shape: (10618, 15)
Shape: (10615, 15)
Shape: (10613, 15)
Shape: (10610, 15)
Shape: (10609, 15)
Shape: (10608, 15)
Shape: (10604, 15)
Shape: (10603, 15)
Shape: (10601, 15)
Shape: (10600, 15)
Shape: (10600, 15)
Shape: (10597, 15)
Shape: (10596, 15)
Test Shape: (2628, 15)
Test Shape: (2827, 15)
Test Shape: (2666, 15)
Test Shape: (2753, 15)
Test Shape: (2681, 15)
Test Shape: (2745, 15)
Test Shape: (2576, 15)
Test Shape: (2719, 15)
Test Shape: (2645, 15)
Test Shape: (2604, 15)
Tes

In [24]:
comp_score(train[train['preds'].notna()]['PRODUCT_LENGTH'].values,train[train['preds'].notna()]['preds'])

(0.35876230163400974, 64.12376983659902)

In [25]:
comp_score(a_train[a_train['preds'].notna()]['PRODUCT_LENGTH'].values,a_train[a_train['preds'].notna()]['preds'])

(0.30912360035729813, 69.08763996427018)

In [26]:
len(d_train)

2249686

In [27]:
len(a_train)

1563838

In [28]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]
def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

temp=d_train.groupby(['BRAND'])['PRODUCT_LENGTH'].agg(['count','min','max'])
temp['diff']=(temp['max']-temp['min'])/temp['min']
brands_smaller_1=temp[(temp['diff']<=1.7)&(temp['count']>7)].index

for i in tqdm(set(brands_smaller_1.tolist()+[1093387,568826,368938])):
    temp=d_train[d_train['BRAND']==i]['PRODUCT_LENGTH'].values
    ind=d_train[d_train['BRAND']==i]['PRODUCT_LENGTH'].index
     
    if(len(ind)==0):
        continue
            
    val2=getVal(temp)
    mape,score2=comp_score(temp,[val2]*len(temp))
    val=val2
    score=score2
        
    if(i==1093387 | i== 568826):
        val=590.55118
        
    if(i==368938):
        val=393.700787
        
    train.loc[(train['BRAND']==i),'preds']=val
    test.loc[(test['BRAND']==i),'preds']=val
    
a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0)

100%|██████████| 9104/9104 [00:55<00:00, 165.33it/s]


In [29]:
# train=train[train['preds'].isna()]
# test=test[test['preds'].isna()]

# def getVal(dist):
#     median = np.power(np.mean(np.power(dist, -1)), -1)
#     return median



# categories= ['watch','clock','print','poster','mixture','soap','supplement','food','protein','vitamin','baby','kid','child','queen','tv','king','copper', 'diamond', 'gold', 'aluminum', 'wood', 'stick', 'bat', 'ball', 'mobile', 'phone', 'cloth', 'book', 'novel', 'guide', 'letter', 'wear', 'men', 'women', 'beauty', 'paper', 'small', 'big', 'xl', 'medium', 'large', 'health', 'suit', 'spring', 'summer', 'winter', 'electronics', 'household', 'food', 'beverage', 'baby', 'pet', 'outdoor', 'improvement', 'tool', 'furniture', 'jewelry', 'watch', 'office', 'supply', 'craft', 'art', 'music', 'vehicle', 'accessory', 'game', 'party', 'decor', 'travel', 'luggage', 'camera', 'photograph', 'security', 'automatic', 'garden', 'plant', 'flower']

# total=0
# for cat in tqdm(categories):
#     temp=train[(train['text'].str.contains(cat))].groupby(['PRODUCT_TYPE_ID'])['PRODUCT_LENGTH'].agg(['min','max','count',getVal])
#     temp['diff']=temp['max']-temp['min']
#     temp=temp[(temp['count']>2)&(temp['diff']<2*temp['min'])]
    
#     values=temp['getVal'].to_dict()
#     train.loc[(train['text'].str.contains(cat))&(train['PRODUCT_TYPE_ID'].isin(values.keys())),'preds']=train.loc[(train['text'].str.contains(cat))&(train['PRODUCT_TYPE_ID'].isin(values.keys())),'PRODUCT_TYPE_ID'].apply(lambda x:values[x])
#     test.loc[(test['text'].str.contains(cat))&(test['PRODUCT_TYPE_ID'].isin(values.keys())),'preds']=test.loc[(test['text'].str.contains(cat))&(test['PRODUCT_TYPE_ID'].isin(values.keys())),'PRODUCT_TYPE_ID'].apply(lambda x:values[x])
#     print(train.shape)
#     print(test.shape)
# a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
# a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0) 

In [30]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]
def get_pred_value(value):
    idx, train, test = value
    for i_,i in enumerate(idx):
        if (i_%1000==0):
            print(f"Done:{i_}/{len(idx)}\n")
        temp=d_train[d_train['PRODUCT_TYPE_ID']==i]['PRODUCT_LENGTH'].values
        ind=d_train[d_train['PRODUCT_TYPE_ID']==i]['PRODUCT_LENGTH'].index

        val2=getVal(temp)
        mape,score2=comp_score(temp,[val2]*len(temp))
    #     preds[ind]=val2
        val=val2
        score=score2
        train.loc[train['PRODUCT_TYPE_ID']==i,'preds']=min(val,1000)
        test.loc[test['PRODUCT_TYPE_ID']==i,'preds']=min(val,1000)
    return (train,test)

def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

label_enc=pd.DataFrame(train['PRODUCT_TYPE_ID'].value_counts())
index=label_enc[label_enc['PRODUCT_TYPE_ID']>0].index.values.tolist()
# create a pool of worker processes
print(len(index))
indexes = split_list(index,multiprocessing.cpu_count())
train_lists = [train.loc[train['PRODUCT_TYPE_ID'].isin(index_)] for index_ in indexes]
test_lists = [test.loc[test['PRODUCT_TYPE_ID'].isin(index_)] for index_ in indexes]
train_ = train.loc[~train['PRODUCT_TYPE_ID'].isin(index)]
test_ = test.loc[~test['PRODUCT_TYPE_ID'].isin(index)]
for i in range(len(indexes)):
    print(f"Shape: {train_lists[i].shape}")

for i in range(len(indexes)):
    print(f"Test Shape: {test_lists[i].shape}")
func_list = [(idx, train_, test_) for (idx, train_, test_) in zip(indexes, train_lists, test_lists)]

if __name__ == '__main__':
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        result = pool.map(get_pred_value, func_list)
    train = pd.concat([result[i][0] for i in range(len(result))], axis = 0, ignore_index = True)
    test = pd.concat([result[i][1] for i in range(len(result))], axis = 0, ignore_index = True)
    train = pd.concat([train, train_], axis = 0, ignore_index = True)
    test = pd.concat([test, test_], axis = 0, ignore_index = True)
    print(train.shape)
    print(test.shape)
    a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
    a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0) 
    del train_, test_, train_lists, test_lists, indexes, func_list
    gc.collect()

12559
Shape: (35718, 15)
Shape: (27041, 15)
Shape: (19663, 15)
Shape: (18502, 15)
Shape: (18314, 15)
Shape: (18117, 15)
Shape: (17581, 15)
Shape: (16577, 15)
Shape: (16453, 15)
Shape: (16253, 15)
Shape: (16162, 15)
Shape: (16045, 15)
Shape: (15958, 15)
Shape: (15839, 15)
Shape: (15681, 15)
Shape: (15491, 15)
Shape: (15312, 15)
Shape: (15250, 15)
Shape: (15201, 15)
Shape: (15102, 15)
Shape: (14949, 15)
Shape: (14878, 15)
Shape: (14783, 15)
Shape: (14735, 15)
Shape: (14577, 15)
Shape: (14537, 15)
Shape: (14468, 15)
Shape: (14414, 15)
Shape: (14365, 15)
Shape: (14268, 15)
Shape: (14123, 15)
Shape: (14085, 15)
Shape: (13972, 15)
Shape: (13913, 15)
Shape: (13824, 15)
Shape: (13777, 15)
Shape: (13720, 15)
Shape: (13688, 15)
Shape: (13630, 15)
Shape: (13555, 15)
Test Shape: (15881, 15)
Test Shape: (12196, 15)
Test Shape: (8829, 15)
Test Shape: (8056, 15)
Test Shape: (8168, 15)
Test Shape: (7899, 15)
Test Shape: (7567, 15)
Test Shape: (7340, 15)
Test Shape: (7359, 15)
Test Shape: (7266, 15)
Te

In [31]:
comp_score(train[train['preds'].notna()]['PRODUCT_LENGTH'].values,train[train['preds'].notna()]['preds'])

(0.833444026894338, 16.655597310566193)

In [32]:
comp_score(a_train[a_train['preds'].notna()]['PRODUCT_LENGTH'].values,a_train[a_train['preds'].notna()]['preds'])

(0.45613592307328044, 54.38640769267196)

In [33]:
train=train[train['preds'].isna()]
test=test[test['preds'].isna()]


hm=getVal(d_train['PRODUCT_LENGTH'])

train['preds'].fillna(hm,inplace=True)
test['preds'].fillna(hm,inplace=True)


a_train=pd.concat([a_train,train[train['preds'].notna()]],axis=0)    
a_test=pd.concat([a_test,test[test['preds'].notna()]],axis=0)    


In [34]:
len(a_test)

734736

In [35]:
a_test['PRODUCT_LENGTH']=a_test['preds']
a_test = a_test.drop_duplicates(subset= ["PRODUCT_ID"], keep='last')
submission=pd.read_csv('/content/data/dataset/sample_submission.csv')
submission.rename(columns={'PRODUCT_LENGTH':'L'},inplace=True)
submission=submission.merge(a_test[['PRODUCT_ID','PRODUCT_LENGTH']],on='PRODUCT_ID')
submission[['PRODUCT_ID','PRODUCT_LENGTH']].to_csv('submission.csv',index=False)

In [36]:
submission

,PRODUCT_ID,L,PRODUCT_LENGTH
0,604373,701.093794,564.430451
1,1729783,734.506163,881.189143
2,1871949,741.360258,1000.000000
3,1107571,730.327767,90.946316
4,624253,666.847946,506.967964
...,...,...,...
734731,921419,733.838809,1000.000000
734732,2456362,746.810825,612.000000
734733,841529,691.127128,116.927131
734734,1190194,757.643591,885.106383
